In [ ]:
!pip install mysql-connector-python
import mysql.connector
from datetime import datetime,timedelta

     |████████████████████████████████| 37.6 MB 1.3 MB/s 


In [ ]:
import getpass
mydb = mysql.connector.connect(
  host=input('host:'),
  user=input('user:'),
  password=getpass.getpass('password:')
)

import json
import pandas as pd
import numpy as np
mycursor = mydb.cursor()

def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x[0])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def exec_sql(sql):
  mycursor.execute(sql)
  myresult = mycursor.fetchall()
  return myresult

def ex_msr_tipe(t):
  return exec_sql('select * from primaprod.master_sumber_referensi where tipe_ref = "'+t+'"')


host:120.29.158.5
user:analis_sirs
password:··········


In [ ]:
def exc_rsp(tF,tT,kP = '',another = ''):
  cond = ' 1=1 ' if kP == '' else ' kode_poli = "'+kP+'" '
  cond += ' AND 1=1 ' if another == '' else ' AND '+another+' '
  rw = exec_sql('SELECT sum(harga+uang_tusla) H\
  , group_concat(keterangan_json separator "++") K\
  , `apotek_resep_pasien`.`no_resep`, \
  concat("[",apotek_resep_pasien.no_rm,"] ",nama_pelanggan) pelanggan\
  ,tgl_resep_created\
  ,apotek_resep_pasien.detail_reg\
  , group_concat(DISTINCT apotek_resep_pasien_detail.user separator "++") U, id_ruang, concat(instalasi_id, ".", id_smf) pl, \
  date_format(tgl_resep_created, "%y%m%d") T, apotek_resep_pasien.id as id, apotek_resep_pasien.* \
FROM primaprod.apotek_resep_pasien \
LEFT JOIN primaprod.`mastercr_no_kamar` ON `mastercr_no_kamar`.`id`=`id_no_kamar`  \
LEFT JOIN primaprod.`poli_kunjungan_pasien` ON `poli_kunjungan_pasien`.`id` = `detail_reg`   \
INNER JOIN primaprod.`apotek_resep_pasien_detail` ON `apotek_resep_pasien`.`no_resep` = `apotek_resep_pasien_detail`.`no_resep`    \
WHERE tgl_resep_created between "'+tF+' 00:00:00" AND "'+tT+' 23:59:59" AND apotek_resep_pasien.no_resep not like "%RE%"   \
AND '+cond+' \
GROUP BY `apotek_resep_pasien`.`no_resep`   \
ORDER BY pelanggan,`apotek_resep_pasien`.`tgl_resep_created`')
  lER = []
  for x in rw:
    js = dict()
    arr = x[1].split("++")
    js = json.loads(arr[0])
    nr = []
    tx = ''
    if 'eresep_ref' in js:
      tx = js['eresep_ref']['no_eresep']
    else:
      tx = "tanpa eresep"
    nr = [tx,x[2],x[3],x[4].strftime("%d/%m %H:%M"),x[5]] 
    lER.append(nr)
  return lER

def exc_ersp(lER):
  rw = exec_sql('select kp,count(kp) from (SELECT if(kode_poli="300",concat("k",id_kamar),kode_poli) kp \
FROM primaprod.eprescription_resep_pasien \
WHERE no_resep in ("'+'","'.join(lER)+'")) a group by kp')
  return rw

def identify_er_kemo(rm,tglODC,totalHariSebelum = 7):
  since = datetime.strptime(tglODC, "%Y-%m-%d") - timedelta(days = totalHariSebelum)
  rw = exec_sql("select tanggal,pendaftaran.no_reg,nama_poli,group_concat(concat(no_resep,'>',eprescription_resep_pasien.status)) \
  ,poli_kunjungan_pasien.id \
  from primaprod.poli_kunjungan_pasien inner join primaprod.pendaftaran using (no_reg) \
inner join primaprod.master_poli on concat(master_poli.id_instalasi,'.',id_poli) = concat(instalasi_id,'.',id_smf) \
inner join primaprod.eprescription_resep_pasien on detail_reg = poli_kunjungan_pasien.id \
where pendaftaran.no_rm = '"+rm+"' and tanggal between '"+since.strftime("%Y-%m-%d")+" 00:00:00' and '"+tglODC+" 00:00:00' \
group by detail_reg order by tanggal desc")
  return rw



In [ ]:
cb = exc_rsp('2022-01-04','2022-01-04','150','')
cb = pd.DataFrame(cb, columns = ['nomor eresep ','nomor resep','identitas pemilik resep','waktu','detailreg'])
cb

In [ ]:

erInKunj = identify_er_kemo('2570290','2022-01-04',100)
dfErInKunj = pd.DataFrame(erInKunj, columns =['tgl kunjungan','no_reg','poli','eresep','detailreg'])
  
dfErInKunj